In [1]:
!pip install gensim


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]


def vectorize(tokens, w2v_model):
    valid_vecs = [w2v_model.wv[word] for word in tokens if word in w2v_model.wv]
    return np.mean(valid_vecs, axis=0) if valid_vecs else np.zeros(w2v_model.vector_size)


sms_data = pd.read_csv('SMSSpamCollection.csv', delimiter='\t', header=None, names=['Label', 'Message'])
sms_data['Tokens'] = sms_data['Message'].apply(preprocess)


w2v_sms = Word2Vec(sms_data['Tokens'], vector_size=300, window=5, min_count=1, workers=4)
sms_data['Vector'] = sms_data['Tokens'].apply(lambda tokens: vectorize(tokens, w2v_sms))


X_sms = np.vstack(sms_data['Vector'].values)
y_sms = sms_data['Label'].map({'ham': 0, 'spam': 1}).values


X_sms_train, X_sms_test, y_sms_train, y_sms_test = train_test_split(X_sms, y_sms, test_size=0.2, random_state=42)
model_sms = LogisticRegression(max_iter=1000, solver='liblinear')
model_sms.fit(X_sms_train, y_sms_train)
preds_sms = model_sms.predict(X_sms_test)
accuracy_sms = accuracy_score(y_sms_test, preds_sms)


print("SMS Spam Detection Accuracy:", accuracy_sms)



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


SMS Spam Detection Accuracy: 0.8663677130044843
